<a href="https://colab.research.google.com/github/TanishqLambhate/Data-Science-Training/blob/pyspark/Pyspark_Refresh_Incremental_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark


In [5]:
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName('Spark Sql Example').getOrCreate()
df_sales=spark.read.format('csv').option('header','true').option('inferSchema','true').load('/content/sample_data/sales_data.csv')
# df_sales.show()

#Apply transformations (if necessary)
df_transformed=df_sales.withColumn('total_sales',df_sales['quantity']*df_sales['price'])

#Full refresh: Partition the data by 'date' and overwrite the existing data
output_path="/content/sample_data/partitioned_data"
df_transformed.write.partitionBy('date').mode('overwrite').parquet(output_path)

#Verify partitioned data
partitioned_df=spark.read.parquet(output_path)
partitioned_df.show()

+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|transaction_id|customer_id| product|quantity|price|         updated_at|total_sales|      date|
+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|             1|        101|  Laptop|       1| 1000|2024-09-01 08:00:00|       1000|2024-09-01|
|             2|        102|   Phone|       2|  500|2024-09-01 09:00:00|       1000|2024-09-01|
|             5|        105|Keyboard|       1|   50|2024-09-03 12:00:00|         50|2024-09-03|
|             6|        106|   Mouse|       3|   30|2024-09-03 13:00:00|         90|2024-09-03|
|             3|        103|  Tablet|       1|  300|2024-09-02 10:00:00|        300|2024-09-02|
|             4|        104| Monitor|       2|  200|2024-09-02 11:00:00|        400|2024-09-02|
+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+



In [7]:
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName('Spark Sql Example').getOrCreate()
df_sales=spark.read.format('csv').option('header','true').option('inferSchema','true').load('/content/sample_data/sales_data.csv')
# df_sales.show()

#Apply transformations (if necessary)
df_transformed_incremental=df_sales.withColumn('total_sales',df_sales['quantity']*df_sales['price'])

# Incremental load Append the new data to the existing partitioned dataset
output_path="/content/sample_data/partitioned_sales_data"
df_transformed_incremental.write.partitionBy('date').mode('append').parquet(output_path)

#Verify partitioned data
partitioned_df=spark.read.parquet(output_path)
partitioned_df.show()

+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|transaction_id|customer_id| product|quantity|price|         updated_at|total_sales|      date|
+--------------+-----------+--------+--------+-----+-------------------+-----------+----------+
|             1|        101|  Laptop|       1| 1000|2024-09-01 08:00:00|       1000|2024-09-01|
|             2|        102|   Phone|       2|  500|2024-09-01 09:00:00|       1000|2024-09-01|
|             1|        101|  Laptop|       1| 1000|2024-09-01 08:00:00|       1000|2024-09-01|
|             2|        102|   Phone|       2|  500|2024-09-01 09:00:00|       1000|2024-09-01|
|             5|        105|Keyboard|       1|   50|2024-09-03 12:00:00|         50|2024-09-03|
|             6|        106|   Mouse|       3|   30|2024-09-03 13:00:00|         90|2024-09-03|
|             5|        105|Keyboard|       1|   50|2024-09-03 12:00:00|         50|2024-09-03|
|             6|        106|   Mouse|   